In [1]:
import numpy as np
from scipy.spatial.distance import cdist, pdist

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

In [2]:
D = 10000
numGestures = 13
numPositions = 8
numEx = 100

In [3]:
def bipolarize(Y):
    X = np.copy(Y)
    X[X > 0] = 1.0
    X[X < 0] = -1.0
    X[X == 0] = np.random.choice([-1.0, 1.0], size=len(X[X == 0]))
    return X

In [ ]:
# prototypeBits = np.round(np.logspace(1,np.log10(5000),20)).astype('int')
# contextBits = np.round(np.logspace(-0.5,np.log10(5000),20)).astype('int')
# exampleBits = np.round(np.logspace(-0.5,np.log10(5000),20)).astype('int')

prototypeBits = np.round(np.linspace(100,5000,20)).astype('int')
contextBits = np.round(np.logspace(0,5000,20)).astype('int')
exampleBits = np.round(np.logspace(0,5000,20)).astype('int')

# prototypeBits = [2000]
# contextBits = [2000]
# exampleBits = [1000]

numIter = 20

prototypeDistances = np.zeros(len(prototypeBits))
contextDistances = np.zeros(len(contextBits))
exampleDistances = np.zeros(len(exampleBits))

for i,NFlipA in enumerate(prototypeBits):
    hv = np.zeros((100,D))
    seed = np.random.choice([-1.0, 1.0], size=D)
    for n in range(100):
        flipIdx = np.random.permutation(D)[:NFlipA]
        flipBits = np.ones(D)
        flipBits[flipIdx] = -1
        hv[n,:] = seed*flipBits
        
    d = cdist(hv,hv,'hamming')
    prototypeDistances[i] = np.mean(d[~np.eye(d.shape[0],dtype=bool)])
    
for i,NFlipA in enumerate(contextBits):
    hv = np.zeros((100,D))
    seed = np.random.choice([-1.0, 1.0], size=D)
    for n in range(100):
        flipIdx = np.random.permutation(D)[:NFlipA]
        flipBits = np.ones(D)
        flipBits[flipIdx] = -1
        hv[n,:] = seed*flipBits
        
    d = cdist(hv,hv,'hamming')
    contextDistances[i] = np.mean(d[~np.eye(d.shape[0],dtype=bool)])
    
for i,NFlipA in enumerate(exampleBits):
    hv = np.zeros((100,D))
    seed = np.random.choice([-1.0, 1.0], size=D)
    for n in range(100):
        flipIdx = np.random.permutation(D)[:NFlipA]
        flipBits = np.ones(D)
        flipBits[flipIdx] = -1
        hv[n,:] = seed*flipBits
        
    d = cdist(hv,hv,'hamming')
    exampleDistances[i] = np.mean(d[~np.eye(d.shape[0],dtype=bool)])


marginOut = np.zeros((len(prototypeBits),len(contextBits),len(exampleBits),numIter))

for ii in tqdm(range(numIter)):
    for i,NFlipA in enumerate(tqdm(prototypeBits)):
        
        allCentroids = np.zeros((numGestures,numPositions,D))
        seed = np.random.choice([-1.0, 1.0], size=D)
        for g in range(numGestures):
            for p in range(numPositions):
                flipIdx = np.random.permutation(D)[:NFlipA]
                flipBits = np.ones(D)
                flipBits[flipIdx] = -1
                allCentroids[g,p,:] = seed*flipBits

#         temp = allCentroids.reshape(-1,D)
#         d = cdist(temp,temp,'hamming')
#         prototypeDistances[i,ii] = np.mean(d[~np.eye(d.shape[0],dtype=bool)])
        
        for j,NFlipB in enumerate(tqdm(contextBits)):            

            contexts = np.zeros((numPositions,D))
            seed = np.random.choice([-1.0, 1.0], size=D)
            for p in range(numPositions):
                flipIdx = np.random.permutation(D)[:NFlipB]
                flipBits = np.ones(D)
                flipBits[flipIdx] = -1
                contexts[p,:] = seed*flipBits
#             if i == 0:
#                 d = cdist(contexts,contexts,'hamming')
#                 contextDistances[j,ii] = np.mean(d[~np.eye(d.shape[0],dtype=bool)])
            
            mappedCentroids = np.zeros(allCentroids.shape)
            for g in range(numGestures):
                for p in range(numPositions):
                    mappedCentroids[g,p,:] = allCentroids[g,p,:] * contexts[p,:]

            prototypes = np.zeros((numGestures,D))
            for g in range(numGestures):
                prototypes[g,:] = bipolarize(np.sum(mappedCentroids[g,:,:],axis=0))
            
            for k,NFlipC in enumerate(tqdm(exampleBits)):
                margins = np.zeros((numGestures,numPositions,numEx))
                for g in range(numGestures):
                    for p in range(numPositions):
                        examples = np.zeros((numEx,D))
                        seed = mappedCentroids[g,p,:]
                        for ex in range(numEx):
                            flipIdx = np.random.permutation(D)[:NFlipC]
                            flipBits = np.ones(D)
                            flipBits[flipIdx] = -1
                            examples[ex,:] = seed*flipBits
#                         if (i == 0) and (j == 0) and (g == 0) and (p == 0):
#                             d = cdist(examples,examples,'hamming')
#                             exampleDistances[k,ii] = np.mean(d[~np.eye(d.shape[0],dtype=bool)])

                        dists = cdist(examples,prototypes,'hamming')
                        b = np.min(np.delete(dists,g,axis=1),axis=1)
                        a = dists[:,g]
                        margins[g,p,:] = (b - a)/(b + a)

                marginOut[i,j,k,ii] = np.mean(margins)

marginOut = np.mean(marginOut,axis=3)

/Users/andy/opt/anaconda3/envs/hdcpar/lib/python3.7/site-packages/numpy/core/function_base.py:274: RuntimeWarning: overflow encountered in power
  return _nx.power(base, y)


In [ ]:
sns.set_context('poster')

# colors = plt.cm.plasma(np.linspace(0,1,len(contextDistances)))
# colors = plt.cm.jet(np.linspace(0,1,len(contextDistances)))
colors = plt.cm.viridis(np.linspace(0,1,len(contextDistances)))

for k in range(len(exampleDistances)):
    f,ax = plt.subplots(figsize=(15,10))
    for i in range(len(contextDistances)):
        ax.plot(prototypeDistances,marginOut[:,i,k],color=colors[i])
    ax.grid(True)
    ax.legend(['%.3f' % c for c in contextDistances],bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=16, title='Context distance')
    ax.set_xlabel('Distance between prototypes')
    ax.set_ylabel('Classifier margin')
    ax.set_title('Within-class distance = %.3f' % exampleDistances[k])